In [1]:
import geojson
import pandas as pd
import numpy as np
import networkx as nx
import time
import csv
import ast
import shapefile as shp
from shapely.geometry import Polygon,shape,MultiPolygon
import shapely.ops
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [2]:
# Import necessary dataframes
nj_precinct_data = pd.read_csv('precinct-data-congress-nj.csv')
keepcolumns = ['GEOID20','District','Total_2020_Total', 'Total_2020_Pres','Dem_2020_Pres','Rep_2020_Pres','White_2020_Total','Hispanic_2020_Total','Black_2020_Total','Asian_2020_Total','Native_2020_Total','Pacific_2020_Total']
nj_precinct_data = nj_precinct_data[keepcolumns]
nj_precinct_data = nj_precinct_data.set_index('GEOID20', drop = False)

#Set this to whatever your seed map is!!!
nj_makeFair = pd.read_csv('test_fairv4.csv')
# Must Set This to previous iteration SEED!!!

nj_makeFair = nj_makeFair.set_index('GEOID20', drop= False)

for index, short in nj_makeFair.iterrows():
    nj_precinct_data.loc[short.name, 'District'] = short['District']

nj_contiguity = pd.read_csv('Contiguity_nj.csv', header=None)
nj_contiguity.columns = ['Precinct','Neighbors']

In [3]:
def isDistrictContiguous(district_num, assignment, contiguity_list, ignore_list=[]):
    ## input:
    ## district_num: the district number
    ## assignment: the assignment from precinct to district
    ## contiguity_list: the list of neighbors for each precinct, from the csv file
    contiguity_list.columns = ['Precinct','Neighbors']
    district_graph = nx.Graph() #creates an empty undirected graph
    district_nodes = assignment[assignment['District']==district_num]['GEOID20'].tolist()
    for i in ignore_list:
        try:
            district_nodes.remove(i)
        except ValueError:
            pass
    district_graph.add_nodes_from(district_nodes)
    for id in district_nodes:
        neighbors = ast.literal_eval(contiguity_list[contiguity_list['Precinct']==id]['Neighbors'].values.tolist()[0])
        # needed to convert string to list because the csv encodes the list as a string
        for neighbor in neighbors:
            if neighbor in district_nodes:
                district_graph.add_edge(id,neighbor)
    return nx.is_connected(district_graph)

In [4]:
#nj_current_assignment.to_csv('nj_longlat_unfairmap_maybeFinal.csv',index=False)
def find_majority_targeted_party(party, border_precincts):
    majority_rep = []
    for pre in border_precincts:
        if party == 'Rep':
            if nj_precinct_data.loc[pre, 'Dem_2020_Pres'] < nj_precinct_data.loc[pre, 'Rep_2020_Pres']:
                majority_rep.append(pre)
        else:
            if nj_precinct_data.loc[pre, 'Dem_2020_Pres'] > nj_precinct_data.loc[pre, 'Rep_2020_Pres']:
                majority_rep.append(pre)
                
    return majority_rep

In [5]:
def find_neighbors2(D1_list, target_district):
    D1_border_precincts = []
    
    for precinct in D1_list:
        neighbors = ast.literal_eval(nj_contiguity[nj_contiguity['Precinct']==precinct]['Neighbors'].values.tolist()[0])
        for neighbor in neighbors:
            if nj_makeFair.loc[nj_makeFair['GEOID20']==neighbor,'District'].values.tolist()[0]==target_district:
                D1_border_precincts.append(precinct)
                break
    
    D1_border_precincts_sorted = sorted(D1_border_precincts,
    key=lambda geo: nj_precinct_data.loc[nj_precinct_data['GEOID20'] == geo, 'Total_2020_Total'].values[0],reverse=True)
    
    return D1_border_precincts

In [6]:
#Starting with Unfair map as seed: Statistics
# 1 heavy Rep Distict [8]
# 3 Competitive, 2 with Rep lean [1D, 6R, 10R]
# 8 Heavy Lean Dem [2, 3, 4, 5, 7, 9, 11, 12]

#Goal:
# To realign for political fairness based on NJ population of 
# 38% Rep Voters, 62% Dem, pick 3 Dem districts with closer margins and
# algorithmically trade precints to align them to Rep districts or competitiveness

# Based on map statistics we've identified our 3 target districts are [7, 4, 2]
# identified adjacent districts with higher populations inwhich to pull rep population from

def political_realign(nj_makeFair, party_target):
    
# Increase Republicans
#     target_dis = {
#         7 :[5, 11, 1],
#         4 :[10],
#         2 :[6, 8]
#     }

# Increase Democrats
    target_dis = {
    5 :[7, 3, 9],
    10 :[4]
    }
    
    #index 0 = Republican, 1 = Dem
    party_balance = []
    
    for OG_District, value in target_dis.items():
        print(OG_District)
        party_balance = [sum(nj_precinct_data[nj_precinct_data['District'] == OG_District]['Rep_2020_Pres']), sum(nj_precinct_data[nj_precinct_data['District'] == OG_District]['Dem_2020_Pres'])]
        print(party_balance)
        
        for Reap_District in value:
            #print(str(OG_District) + " : " + str(Reap_District))
            #Create list for bordering districts with targeted political population to add to OG_District
            D1_list = nj_makeFair[nj_makeFair['District']== Reap_District]['GEOID20']
            D1_border_precincts = find_neighbors2(D1_list, OG_District)
            Party_Pre_List = find_majority_targeted_party(party_target, D1_border_precincts)
            #Add them if contiguous and until party is bigger
            for pre in Party_Pre_List:
                if party_target == 'Rep':
                    if (party_balance[0] + nj_precinct_data.loc[pre, 'Rep_2020_Pres']) < party_balance[1]:
                        nj_precinct_data.loc[pre, 'District'] = OG_District
                        nj_makeFair.loc[pre, 'District'] = OG_District
                else:
                    if (party_balance[1] + nj_precinct_data.loc[pre, 'Dem_2020_Pres']) < party_balance[0]:
                        nj_precinct_data.loc[pre, 'District'] = OG_District
                        nj_makeFair.loc[pre, 'District'] = OG_District
                        
                if not (isDistrictContiguous(Reap_District, nj_makeFair, nj_contiguity) and
                        isDistrictContiguous(OG_District, nj_makeFair, nj_contiguity)):
                    # Revert if contiguity is broken
                    nj_precinct_data.loc[pre, 'District'] = Reap_District
                    nj_makeFair.loc[pre, 'District'] = Reap_District
                    print('Broke Congtiguous. Reverting: ' + str(pre))
                    continue
            
                if party_target == 'Rep':
                    party_balance[0] = party_balance[0] + nj_precinct_data.loc[pre, 'Rep_2020_Pres']
                else:
                    party_balance[1] = party_balance[1] + nj_precinct_data.loc[pre, 'Dem_2020_Pres']
                print('Traded ' + str(pre) + ": " + str(party_balance[0]) + " : " + str(party_balance[1]))

In [9]:
# Rep for target to increase republican, Dem for demoracts
# nj_makeFair is the seed map import. If you do iterations, make sure to up this to newer map
#political_realign(nj_makeFair, 'Dem')

5
[118906, 214910]
Traded 34041080006: 118906 : 215349
Traded 34003230015: 118906 : 215941
Traded 34003085003: 118906 : 216332
Traded 34003085009: 118906 : 216810
Traded 34003285001: 118906 : 217143
Traded 34003105503: 118906 : 217401
Traded 34003105501: 118906 : 218023
Traded 34003325003: 118906 : 218525
Traded 34003295001: 118906 : 219100
Traded 34003105301: 118906 : 219687
Traded 34003105302: 118906 : 220227
Traded 34027135021: 118906 : 220903
Traded 34039060211: 118906 : 221482
Traded 34039060107: 118906 : 221923
Traded 34013010313: 118906 : 222354
Traded 34013010311: 118906 : 222693
Traded 34013010312: 118906 : 223118
Traded 34039095036: 118906 : 223520
Traded 34039095025: 118906 : 223792
Traded 34039015028: 118906 : 224077
Traded 34013005102: 118906 : 224691
Traded 34013005103: 118906 : 225130
Traded 34013010310: 118906 : 225542
Traded 34039100204: 118906 : 226102
Traded 34039100206: 118906 : 226549
Traded 34039100205: 118906 : 227132
Traded 34039100201: 118906 : 227619
Traded 34

Traded 34013110108: 118906 : 320909
Traded 34031010115: 118906 : 321508
Traded 34013010211: 118906 : 321781
Traded 34013010209: 118906 : 322350
Traded 34013010206: 118906 : 323034
Traded 34039005001: 118906 : 323602
Traded 34039085009: 118906 : 324183
Traded 34039085008: 118906 : 324498
Traded 34039050008: 118906 : 324725
Traded 34039085001: 118906 : 325160
Traded 34013055002: 118906 : 325999
Traded 34013060007: 118906 : 326756
Traded 34027085011: 118906 : 327197
Traded 34027085004: 118906 : 327630
Traded 34013027N03: 118906 : 328472
Traded 34013027N04: 118906 : 328861
Traded 34013027E01: 118906 : 329748
Traded 34013060006: 118906 : 330331
Traded 34013060002: 118906 : 331048
Traded 34013110107: 118906 : 331393
Traded 34013110106: 118906 : 332131
Traded 34027085009: 118906 : 332547
Traded 34027085005: 118906 : 332962
Traded 34027085001: 118906 : 333388
Traded 34013095006: 118906 : 334071
Traded 34039085010: 118906 : 334603
Traded 34013095003: 118906 : 335263
Traded 34039085012: 118906 :

In [8]:
#nj_makeFair.to_csv('test_fairv5.csv',index=False)